In [9]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

from fastai.learner import *

import torchtext
from torchtext import vocab, data
from torchtext.datasets import language_modeling

from fastai.rnn_reg import *
from fastai.rnn_train import *
from fastai.nlp import *
from fastai.lm_rnn import *

import dill as pickle
import spacy


PATH='/home/wk/myProjects/data/Enron/oper/'

TRN_PATH = 'train/'
VAL_PATH = 'test/'
TRN = f'{PATH}{TRN_PATH}'
VAL = f'{PATH}{VAL_PATH}'

bs=32; bptt=75
em_sz = 300  # size of each embedding vector
nh = 500     # number of hidden activations per layer
nl = 3       # number of layers

FILES = dict(train=TRN_PATH, validation=VAL_PATH, test=VAL_PATH)
TEXT = pickle.load(open(f'{PATH}models/TEXT.pkl','rb'))

md = LanguageModelData.from_text_files(PATH, TEXT, **FILES, bs=bs, bptt=bptt, min_freq=10)

opt_fn = partial(optim.Adam, betas=(0.7, 0.99))
learner = md.get_model(opt_fn, em_sz, nh, nl,
               dropouti=0.05, dropout=0.05, wdrop=0.1, dropoute=0.02, dropouth=0.05)
learner.reg_fn = partial(seq2seq_reg, alpha=2, beta=1)
learner.clip=0.3

learner.load_encoder('adam3_10_enc')

In [10]:
#PATH=Path('/home/wk/myProjects/data/Enron/tag/')

In [11]:
#[str(i) for i in PATH.iterdir()]
#PATH/'here.txt'.open()

In [12]:
### Test
ss=r"""
Message-ID: <32399675.1075856463241.JavaMail.evans@thyme>
Date: Fri, 22 Dec 2000 09:58:00 -0800 (PST)
From: vince.kaminski@enron.com
To: tanya.tamarchenko@enron.com
Subject: Re: Suggestion: implementing VAR based on non-normal log-returns
 simulations
Mime-Version: 1.0
Content-Type: text/plain; charset=us-ascii
Content-Transfer-Encoding: 7bit
X-From: Vince J Kaminski
X-To: Tanya Tamarchenko
X-cc: 
X-bcc: 
X-Folder: \Vincent_Kaminski_Jun2001_3\Notes Folders\Sent
X-Origin: Kaminski-V
X-FileName: vkamins.nsf

Tanya,

I could not locate the file.

Vince





Tanya Tamarchenko
12/07/2000 01:17 PM
To: Vince J Kaminski/HOU/ECT@ECT, Rabi De/NA/Enron@ENRON, Jaesoo 
Lew/NA/Enron@ENRON
cc:  
Subject: Re: Suggestion: implementing VAR based on non-normal log-returns 
simulations  

Everybody, 
we were talking for a while about using non-normal distributions in the 
Monte-Carlo simulations in our VAR model.
I put together some suggestion regarding this. The text is under 
O:\_Dropbox\Tanya\non_normal_logs.doc

Look through this 3 page document, and let me know what you think, please.


Tanya



"""
s = [TEXT.preprocess(ss)] 
t=TEXT.numericalize(s)
' '.join(s[0])

'message-id: <32399675.1075856463241.javamail.evans@thyme> date: fri, 22 dec 2000 09:58:00 -0800 (pst) from: vince.kaminski@enron.com to: tanya.tamarchenko@enron.com subject: re: suggestion: implementing var based on non-normal log-returns simulations mime-version: 1.0 content-type: text/plain; charset=us-ascii content-transfer-encoding: 7bit x-from: vince j kaminski x-to: tanya tamarchenko x-cc: x-bcc: x-folder: \\vincent_kaminski_jun2001_3\\notes folders\\sent x-origin: kaminski-v x-filename: vkamins.nsf tanya, i could not locate the file. vince tanya tamarchenko 12/07/2000 01:17 pm to: vince j kaminski/hou/ect@ect, rabi de/na/enron@enron, jaesoo lew/na/enron@enron cc: subject: re: suggestion: implementing var based on non-normal log-returns simulations everybody, we were talking for a while about using non-normal distributions in the monte-carlo simulations in our var model. i put together some suggestion regarding this. the text is under o:\\_dropbox\\tanya\\non_normal_logs.doc loo

In [13]:
import copy
ss = copy.deepcopy(s)
s_p = [ss[0][0:3]]
t=TEXT.numericalize(s)
#t = t.long().cpu()

m=learner.model

m[0].bs=1
m.eval()
m.reset()

#m.cpu()
#m.cuda()
res,*_ = m(t)

nexts = torch.topk(res[-1], 10)[1]

for i in range(len(s[0])-3):
    n=res[-1].topk(2)[1]    
    predict = TEXT.vocab.itos[n.data[0]]    
    if (predict[0:2] == '@@'):
        print("pos=" + str(i+3) + "; " + predict + " <- " + s[0][i+3] , end=' \n')                    
        ss[0][i+3]=predict
    
    s_p = [ss[0][0:i+4]]
    t=TEXT.numericalize(s_p)

    res,*_ = m(t)    
    
m[0].bs=bs       
print('...')

pos=4; @@othr_dt@@ <- 22 
pos=5; @@othr_tm@@ <- dec 
pos=11; @@email@@ <- vince.kaminski@enron.com 
pos=13; @@othr_em@@ <- tanya.tamarchenko@enron.com 
pos=32; @@firstname@@ <- vince 
pos=33; @@lastname@@ <- j 
pos=57; @@othr_dt@@ <- 12/07/2000 
pos=58; @@othr_tm@@ <- 01:17 
pos=61; @@firstname@@ <- vince 
...


In [14]:
#https://askubuntu.com/questions/607118/cuda-not-working-after-returning-laptop-from-sleep#750939
len(s[0])

125

In [15]:
#sudo rmmod nvidia_uvm
#sudo modprobe nvidia_uvm

In [16]:
#t.long().cpu()